In [90]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import pandas as pd
import zipfile

import requests
from io import BytesIO

# opening and parse EDGAR Log File Data Set html
edgarLogUrl = 'https://www.sec.gov/dera/data/edgar-log-file-data-set.html'
edgarLogPage = BeautifulSoup(urlopen(edgarLogUrl),'lxml')

# get log file html of specified year
year = '2003'
try:
    for link in edgarLogPage.findAll('a'):
        if link.text == year:
            edgarLogFiles = BeautifulSoup(urlopen('https://www.sec.gov' + link.get('href')),'lxml')
            break
except Exception:
    print('Error, no', year,'log file data set found！')

# get url of each month
monthList=[]
for i, link in enumerate(edgarLogFiles.findAll('a')):
    if(re.match(r'.*01.zip$',link.text)):
        monthList.insert(12-i,link.get('href'))

# download zip and parse csv file
content = requests.get(monthList[9])
zf = zipfile.ZipFile(BytesIO(content.content))
for name in zf.namelist():
    if (re.match(r'.*.csv$',name)):
        df = pd.read_csv(zf.open(name)) 

df.info()
df.isnull().sum()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119840 entries, 0 to 119839
Data columns (total 15 columns):
ip           119840 non-null object
date         119840 non-null object
time         119840 non-null object
zone         119840 non-null float64
cik          119840 non-null float64
accession    119840 non-null object
extention    119840 non-null object
code         119840 non-null float64
size         102490 non-null float64
idx          119840 non-null float64
norefer      119840 non-null float64
noagent      119840 non-null float64
find         119840 non-null float64
crawler      119840 non-null float64
browser      95944 non-null object
dtypes: float64(9), object(6)
memory usage: 13.7+ MB


,zone,cik,code,size,idx,norefer,noagent,find,crawler
count,119840.0,1.198400e+05,119840.000000,1.024900e+05,119840.000000,119840.000000,119840.000000,119840.000000,119840.000000
mean,400.0,8.085589e+05,215.585614,1.406973e+05,0.394351,0.310664,0.014603,3.342874,0.042690
std,0.0,3.929611e+05,37.675826,5.280427e+05,0.488713,0.462767,0.119957,3.906975,0.202159
min,400.0,2.000000e+01,200.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,400.0,7.327120e+05,200.000000,2.870000e+03,0.000000,0.000000,0.000000,0.000000,0.000000
50%,400.0,9.215030e+05,200.000000,5.548000e+03,0.000000,0.000000,0.000000,1.000000,0.000000
75%,400.0,1.089567e+06,200.000000,4.355150e+04,1.000000,1.000000,0.000000,9.000000,0.000000
max,400.0,1.265723e+06,416.000000,6.428835e+07,1.000000,1.000000,1.000000,10.000000,1.000000


In [82]:
df.head(10)

,ip,date,time,zone,cik,accession,extention,code,size,idx,norefer,noagent,find,crawler,browser
0,129.105.133.bcf,2003-10-01,00:00:23,400.0,900405.0,0000950134-02-001349,.txt,200.0,7702.0,0.0,1.0,0.0,0.0,1.0,NaN
1,129.105.133.bcf,2003-10-01,00:00:23,400.0,891024.0,0001045969-02-000262,.txt,200.0,8675.0,0.0,1.0,0.0,0.0,1.0,NaN
2,129.105.133.bcf,2003-10-01,00:00:47,400.0,893949.0,0001047469-02-002139,.txt,200.0,7377.0,0.0,1.0,0.0,0.0,1.0,NaN
3,164.164.89.djf,2003-10-01,00:01:37,400.0,802681.0,0001181431-03-024733,-index.htm,200.0,2726.0,1.0,0.0,0.0,1.0,0.0,win
4,12.163.71.fdh,2003-10-01,00:01:38,400.0,54058.0,0000897069-03-000996,-index.htm,200.0,2379.0,1.0,0.0,0.0,1.0,0.0,win
5,164.164.89.djf,2003-10-01,00:01:41,400.0,802681.0,0001181431-03-024732,-index.htm,200.0,2534.0,1.0,0.0,0.0,1.0,0.0,win
6,164.164.89.djf,2003-10-01,00:01:45,400.0,802681.0,0001181431-03-024732,xslF345X02/rrd19378.xml,200.0,17730.0,0.0,0.0,0.0,9.0,0.0,win
7,205.188.209.bah,2003-10-01,00:02:07,400.0,801898.0,0000801898-03-000036,thirdqtrtenq.htm,200.0,228808.0,0.0,0.0,0.0,9.0,0.0,win
8,129.105.133.bcf,2003-10-01,00:02:57,400.0,810717.0,0000810717-02-000040,.txt,200.0,18040.0,0.0,1.0,0.0,0.0,1.0,NaN
9,129.105.133.bcf,2003-10-01,00:03:29,400.0,78890.0,0001005477-02-000642,.txt,200.0,13182.0,0.0,1.0,0.0,0.0,1.0,NaN


In [83]:
df['browser'].fillna('unknown', inplace = True)

In [84]:
df.isnull().sum()

ip               0
date             0
time             0
zone             0
cik              0
accession        0
extention        0
code             0
size         17350
idx              0
norefer          0
noagent          0
find             0
crawler          0
browser          0
dtype: int64

In [86]:
df['size'].fillna(df['size'].mean(), inplace = True)

In [87]:
df.isnull().sum()

ip           0
date         0
time         0
zone         0
cik          0
accession    0
extention    0
code         0
size         0
idx          0
norefer      0
noagent      0
find         0
crawler      0
browser      0
dtype: int64

In [89]:
df.describe()

,zone,cik,code,size,idx,norefer,noagent,find,crawler
count,119840.0,1.198400e+05,119840.000000,1.198400e+05,119840.000000,119840.000000,119840.000000,119840.000000,119840.000000
mean,400.0,8.085589e+05,215.585614,1.406973e+05,0.394351,0.310664,0.014603,3.342874,0.042690
std,0.0,3.929611e+05,37.675826,4.883246e+05,0.488713,0.462767,0.119957,3.906975,0.202159
min,400.0,2.000000e+01,200.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,400.0,7.327120e+05,200.000000,3.044000e+03,0.000000,0.000000,0.000000,0.000000,0.000000
50%,400.0,9.215030e+05,200.000000,9.021500e+03,0.000000,0.000000,0.000000,1.000000,0.000000
75%,400.0,1.089567e+06,200.000000,1.406973e+05,1.000000,1.000000,0.000000,9.000000,0.000000
max,400.0,1.265723e+06,416.000000,6.428835e+07,1.000000,1.000000,1.000000,10.000000,1.000000
